## 151 Cumsum @month_lag

In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features
import datetime

from tqdm import tqdm
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import time
import sys
import datetime
from sklearn.metrics import mean_squared_error
from mv_wg_avg import exp_weight_avg


os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

### Data Load

In [55]:
def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
df_train.set_index(key, inplace=True)
df_test.set_index(key, inplace=True)
df_hist = utils.read_df_pkl('../input/hist*0*')
df_new = utils.read_df_pkl('../input/new*0*')

df_train = reduce_mem_usage(df_train)
df_test  = reduce_mem_usage(df_test )
df_hist  = reduce_mem_usage(df_hist )
df_new   = reduce_mem_usage(df_new  )

100%|██████████| 3/3 [00:00<00:00, 172.59it/s]


SystemExit: 

In [5]:
df = df_new
df.sort_values(by=[key, 'purchase_date'], inplace=True)
df['authorized_flag'] =  df['authorized_flag'].map(lambda x: 0 if x=='N' else 1)
df['category_1'] =  df['category_1'].map(lambda x: 0 if x=='N' else 1)
# df['yyyymm'] = df['purchase_date'].map(lambda x: str(x)[:7])
df['purchase_date'] = pd.to_datetime(df['purchase_date'])
df['purchase_date_shift1'] =  df.groupby(key)['purchase_date'].shift(1)
df['diff_days'] =  (df['purchase_date'] - df['purchase_date_shift1']).dt.days
df = df[[key, 'month_lag', 'authorized_flag', 'category_1', 'purchase_amount', 'installments', 'diff_days']]
df_new = df.copy()
del df
gc.collect()

df = df_hist
df.sort_values(by=[key, 'purchase_date'], inplace=True)
df['authorized_flag'] =  df['authorized_flag'].map(lambda x: 0 if x=='N' else 1)
df['category_1'] =  df['category_1'].map(lambda x: 0 if x=='N' else 1)
df['purchase_date'] = pd.to_datetime(df['purchase_date'])
# df['yyyymm'] = df['purchase_date'].map(lambda x: str(x)[:7])
df['purchase_date_shift1'] =  df.groupby(key)['purchase_date'].shift(1)
df['diff_days'] =  (df['purchase_date'] - df['purchase_date_shift1']).dt.days
df = df[[key, 'month_lag', 'authorized_flag', 'category_1', 'purchase_amount', 'installments', 'diff_days']]
df_hist = df.copy()
del df
gc.collect()
df_hist.head()

KeyError: 'purchase_date'

In [8]:
df_trans = pd.concat([df_hist, df_new], axis=0)
df_trans = reduce_mem_usage(df_trans)
auth1 = df_trans[df_trans['authorized_flag']==1]
auth0 = df_trans[df_trans['authorized_flag']==0]

Mem. usage decreased to 800.17 Mb (44.9% reduction)


### 全month_lagを全IDにもたせ、cumsumする

In [19]:
# auth1
df = auth1.copy()
fname = 'auth1'
df['record'] = 1

aggs = {}
aggs['category_1'] = ['mean', 'sum']
aggs['purchase_amount'] = ['mean', 'sum']
aggs['diff_days'] = ['mean']
aggs['installments'] = ['mean']
aggs['record'] = ['sum']

# 値が大きいほど、直近の数字となる -> 逆にしなきゃ
df['month_lag'] =  df['month_lag'] - df['month_lag'].min()
tmp = df.groupby([key, 'month_lag']).agg(aggs)
new_columns = get_new_columns(fname, aggs)
tmp.columns = new_columns

tmp  = reduce_mem_usage(tmp).reset_index()

label = 'month_lag'
df_list = []
ignore_cols = [key, label]
base = tmp[[key, label]].drop_duplicates()
base['tmp'] = 0
base = base.set_index([key, label]).unstack()
base = base.fillna(0).stack().reset_index()[[key, label]]
tmp = base.merge(tmp, how='left', on=[key, label]).fillna(0)

for col in tqdm(tmp.columns):
    if col in ignore_cols:continue
    tmp_feat = pd.concat([tmp[[key, label]], tmp.groupby(key)[col].cumsum()], axis=1)
    tmp_feat = tmp_feat.pivot_table(index=key, columns=label, values=col)
    tmp_feat.columns = [f"{label}_{c}_cumsum_{col}" for c in tmp_feat.columns]
    df_list.append(tmp_feat)
    
df_auth1_wg = pd.concat(df_list, axis=1)
df_auth1_wg.head()

,month_lag_0_cumsum_auth1_category_1_mean,month_lag_1_cumsum_auth1_category_1_mean,month_lag_2_cumsum_auth1_category_1_mean,month_lag_3_cumsum_auth1_category_1_mean,month_lag_4_cumsum_auth1_category_1_mean,month_lag_5_cumsum_auth1_category_1_mean,month_lag_6_cumsum_auth1_category_1_mean,month_lag_7_cumsum_auth1_category_1_mean,month_lag_8_cumsum_auth1_category_1_mean,month_lag_9_cumsum_auth1_category_1_mean,month_lag_10_cumsum_auth1_category_1_mean,month_lag_11_cumsum_auth1_category_1_mean,month_lag_12_cumsum_auth1_category_1_mean,month_lag_13_cumsum_auth1_category_1_mean,month_lag_14_cumsum_auth1_category_1_mean,month_lag_15_cumsum_auth1_category_1_mean,month_lag_0_cumsum_auth1_category_1_sum,month_lag_1_cumsum_auth1_category_1_sum,month_lag_2_cumsum_auth1_category_1_sum,month_lag_3_cumsum_auth1_category_1_sum,month_lag_4_cumsum_auth1_category_1_sum,month_lag_5_cumsum_auth1_category_1_sum,month_lag_6_cumsum_auth1_category_1_sum,month_lag_7_cumsum_auth1_category_1_sum,month_lag_8_cumsum_auth1_category_1_sum,month_lag_9_cumsum_auth1_category_1_sum,month_lag_10_cumsum_auth1_category_1_sum,month_lag_11_cumsum_auth1_category_1_sum,month_lag_12_cumsum_auth1_category_1_sum,month_lag_13_cumsum_auth1_category_1_sum,month_lag_14_cumsum_auth1_category_1_sum,month_lag_15_cumsum_auth1_category_1_sum,month_lag_0_cumsum_auth1_purchase_amount_mean,month_lag_1_cumsum_auth1_purchase_amount_mean,month_lag_2_cumsum_auth1_purchase_amount_mean,month_lag_3_cumsum_auth1_purchase_amount_mean,month_lag_4_cumsum_auth1_purchase_amount_mean,month_lag_5_cumsum_auth1_purchase_amount_mean,month_lag_6_cumsum_auth1_purchase_amount_mean,month_lag_7_cumsum_auth1_purchase_amount_mean,month_lag_8_cumsum_auth1_purchase_amount_mean,month_lag_9_cumsum_auth1_purchase_amount_mean,month_lag_10_cumsum_auth1_purchase_amount_mean,month_lag_11_cumsum_auth1_purchase_amount_mean,month_lag_12_cumsum_auth1_purchase_amount_mean,month_lag_13_cumsum_auth1_purchase_amount_mean,month_lag_14_cumsum_auth1_purchase_amount_mean,month_lag_15_cumsum_auth1_purchase_amount_mean,month_lag_0_cumsum_auth1_purchase_amount_sum,month_lag_1_cumsum_auth1_purchase_amount_sum,month_lag_2_cumsum_auth1_purchase_amount_sum,month_lag_3_cumsum_auth1_purchase_amount_sum,month_lag_4_cumsum_auth1_purchase_amount_sum,month_lag_5_cumsum_auth1_purchase_amount_sum,month_lag_6_cumsum_auth1_purchase_amount_sum,month_lag_7_cumsum_auth1_purchase_amount_sum,month_lag_8_cumsum_auth1_purchase_amount_sum,month_lag_9_cumsum_auth1_purchase_amount_sum,month_lag_10_cumsum_auth1_purchase_amount_sum,month_lag_11_cumsum_auth1_purchase_amount_sum,month_lag_12_cumsum_auth1_purchase_amount_sum,month_lag_13_cumsum_auth1_purchase_amount_sum,month_lag_14_cumsum_auth1_purchase_amount_sum,month_lag_15_cumsum_auth1_purchase_amount_sum,month_lag_0_cumsum_auth1_diff_days_mean,month_lag_1_cumsum_auth1_diff_days_mean,month_lag_2_cumsum_auth1_diff_days_mean,month_lag_3_cumsum_auth1_diff_days_mean,month_lag_4_cumsum_auth1_diff_days_mean,month_lag_5_cumsum_auth1_diff_days_mean,month_lag_6_cumsum_auth1_diff_days_mean,month_lag_7_cumsum_auth1_diff_days_mean,month_lag_8_cumsum_auth1_diff_days_mean,month_lag_9_cumsum_auth1_diff_days_mean,month_lag_10_cumsum_auth1_diff_days_mean,month_lag_11_cumsum_auth1_diff_days_mean,month_lag_12_cumsum_auth1_diff_days_mean,month_lag_13_cumsum_auth1_diff_days_mean,month_lag_14_cumsum_auth1_diff_days_mean,month_lag_15_cumsum_auth1_diff_days_mean,month_lag_0_cumsum_auth1_installments_mean,month_lag_1_cumsum_auth1_installments_mean,month_lag_2_cumsum_auth1_installments_mean,month_lag_3_cumsum_auth1_installments_mean,month_lag_4_cumsum_auth1_installments_mean,month_lag_5_cumsum_auth1_installments_mean,month_lag_6_cumsum_auth1_installments_mean,month_lag_7_cumsum_auth1_installments_mean,month_lag_8_cumsum_auth1_installments_mean,month_lag_9_cumsum_auth1_installments_mean,month_lag_10_cumsum_auth1_installments_mean,month_lag_11_cumsum_auth1_installments_mean,month_lag_12_cumsum_auth1_installments_mean,month_lag_13_cu

In [20]:
# auth0
df = auth0.copy()
fname = 'auth0'
df['record'] = 1

aggs = {}
aggs['category_1'] = ['mean', 'sum']
aggs['purchase_amount'] = ['mean', 'sum']
aggs['diff_days'] = ['mean']
aggs['installments'] = ['mean']
aggs['record'] = ['sum']

# 値が大きいほど、直近の数字となる -> 逆にしなきゃ
df['month_lag'] =  df['month_lag'] - df['month_lag'].min()
tmp = df.groupby([key, 'month_lag']).agg(aggs)
new_columns = get_new_columns(fname, aggs)
tmp.columns = new_columns

tmp  = reduce_mem_usage(tmp).reset_index()

label = 'month_lag'
df_list = []
ignore_cols = [key, label]
base = tmp[[key, label]].drop_duplicates()
base['tmp'] = 0
base = base.set_index([key, label]).unstack()
base = base.fillna(0).stack().reset_index()[[key, label]]
tmp = base.merge(tmp, how='left', on=[key, label]).fillna(0)

for col in tqdm(tmp.columns):
    if col in ignore_cols:continue
    tmp_feat = pd.concat([tmp[[key, label]], tmp.groupby(key)[col].cumsum()], axis=1)
    tmp_feat = tmp_feat.pivot_table(index=key, columns=label, values=col)
    tmp_feat.columns = [f"{label}_{c}_cumsum_{col}" for c in tmp_feat.columns]
    df_list.append(tmp_feat)
    
df_auth0_wg = pd.concat(df_list, axis=1)
df_auth0_wg.head()

Mem. usage decreased to 23.40 Mb (41.6% reduction)


100%|██████████| 9/9 [00:18<00:00,  2.46s/it]


,month_lag_0_cumsum_auth0_category_1_mean,month_lag_1_cumsum_auth0_category_1_mean,month_lag_2_cumsum_auth0_category_1_mean,month_lag_3_cumsum_auth0_category_1_mean,month_lag_4_cumsum_auth0_category_1_mean,month_lag_5_cumsum_auth0_category_1_mean,month_lag_6_cumsum_auth0_category_1_mean,month_lag_7_cumsum_auth0_category_1_mean,month_lag_8_cumsum_auth0_category_1_mean,month_lag_9_cumsum_auth0_category_1_mean,month_lag_10_cumsum_auth0_category_1_mean,month_lag_11_cumsum_auth0_category_1_mean,month_lag_12_cumsum_auth0_category_1_mean,month_lag_13_cumsum_auth0_category_1_mean,month_lag_0_cumsum_auth0_category_1_sum,month_lag_1_cumsum_auth0_category_1_sum,month_lag_2_cumsum_auth0_category_1_sum,month_lag_3_cumsum_auth0_category_1_sum,month_lag_4_cumsum_auth0_category_1_sum,month_lag_5_cumsum_auth0_category_1_sum,month_lag_6_cumsum_auth0_category_1_sum,month_lag_7_cumsum_auth0_category_1_sum,month_lag_8_cumsum_auth0_category_1_sum,month_lag_9_cumsum_auth0_category_1_sum,month_lag_10_cumsum_auth0_category_1_sum,month_lag_11_cumsum_auth0_category_1_sum,month_lag_12_cumsum_auth0_category_1_sum,month_lag_13_cumsum_auth0_category_1_sum,month_lag_0_cumsum_auth0_purchase_amount_mean,month_lag_1_cumsum_auth0_purchase_amount_mean,month_lag_2_cumsum_auth0_purchase_amount_mean,month_lag_3_cumsum_auth0_purchase_amount_mean,month_lag_4_cumsum_auth0_purchase_amount_mean,month_lag_5_cumsum_auth0_purchase_amount_mean,month_lag_6_cumsum_auth0_purchase_amount_mean,month_lag_7_cumsum_auth0_purchase_amount_mean,month_lag_8_cumsum_auth0_purchase_amount_mean,month_lag_9_cumsum_auth0_purchase_amount_mean,month_lag_10_cumsum_auth0_purchase_amount_mean,month_lag_11_cumsum_auth0_purchase_amount_mean,month_lag_12_cumsum_auth0_purchase_amount_mean,month_lag_13_cumsum_auth0_purchase_amount_mean,month_lag_0_cumsum_auth0_purchase_amount_sum,month_lag_1_cumsum_auth0_purchase_amount_sum,month_lag_2_cumsum_auth0_purchase_amount_sum,month_lag_3_cumsum_auth0_purchase_amount_sum,month_lag_4_cumsum_auth0_purchase_amount_sum,month_lag_5_cumsum_auth0_purchase_amount_sum,month_lag_6_cumsum_auth0_purchase_amount_sum,month_lag_7_cumsum_auth0_purchase_amount_sum,month_lag_8_cumsum_auth0_purchase_amount_sum,month_lag_9_cumsum_auth0_purchase_amount_sum,month_lag_10_cumsum_auth0_purchase_amount_sum,month_lag_11_cumsum_auth0_purchase_amount_sum,month_lag_12_cumsum_auth0_purchase_amount_sum,month_lag_13_cumsum_auth0_purchase_amount_sum,month_lag_0_cumsum_auth0_diff_days_mean,month_lag_1_cumsum_auth0_diff_days_mean,month_lag_2_cumsum_auth0_diff_days_mean,month_lag_3_cumsum_auth0_diff_days_mean,month_lag_4_cumsum_auth0_diff_days_mean,month_lag_5_cumsum_auth0_diff_days_mean,month_lag_6_cumsum_auth0_diff_days_mean,month_lag_7_cumsum_auth0_diff_days_mean,month_lag_8_cumsum_auth0_diff_days_mean,month_lag_9_cumsum_auth0_diff_days_mean,month_lag_10_cumsum_auth0_diff_days_mean,month_lag_11_cumsum_auth0_diff_days_mean,month_lag_12_cumsum_auth0_diff_days_mean,month_lag_13_cumsum_auth0_diff_days_mean,month_lag_0_cumsum_auth0_installments_mean,month_lag_1_cumsum_auth0_installments_mean,month_lag_2_cumsum_auth0_installments_mean,month_lag_3_cumsum_auth0_installments_mean,month_lag_4_cumsum_auth0_installments_mean,month_lag_5_cumsum_auth0_installments_mean,month_lag_6_cumsum_auth0_installments_mean,month_lag_7_cumsum_auth0_installments_mean,month_lag_8_cumsum_auth0_installments_mean,month_lag_9_cumsum_auth0_installments_mean,month_lag_10_cumsum_auth0_installments_mean,month_lag_11_cumsum_auth0_installments_mean,month_lag_12_cumsum_auth0_installments_mean,month_lag_13_cumsum_auth0_installments_mean,month_lag_0_cumsum_auth0_record_sum,month_lag_1_cumsum_auth0_record_sum,month_lag_2_cumsum_auth0_record_sum,month_lag_3_cumsum_auth0_record_sum,month_lag_4_cumsum_auth0_record_sum,month_lag_5_cumsum_auth0_record_sum,month_lag_6_cumsum_auth0_record_sum,month_lag_7_cumsum_auth0_record_sum,month_lag_8_cumsum_auth0_record_sum,month_lag_9_cumsum_auth0_record_sum,month_lag_10_cumsum_auth0_record_sum,month_la

In [21]:
for df in [df_auth1_wg, df_auth0_wg]:
    df_train = df_train.join(df)
    df_test = df_test.join(df)
for col in df_train.columns:
    df_train.rename(columns={col:col.replace('.', '_')}, inplace=True)
    df_test.rename(columns={col:col.replace('.', '_')}, inplace=True)
print(df_train.shape)
display(df_train.head())

(201917, 215)


,first_active_month,feature_1,feature_2,feature_3,target,month_lag_0_cumsum_auth1_category_1_mean,month_lag_1_cumsum_auth1_category_1_mean,month_lag_2_cumsum_auth1_category_1_mean,month_lag_3_cumsum_auth1_category_1_mean,month_lag_4_cumsum_auth1_category_1_mean,month_lag_5_cumsum_auth1_category_1_mean,month_lag_6_cumsum_auth1_category_1_mean,month_lag_7_cumsum_auth1_category_1_mean,month_lag_8_cumsum_auth1_category_1_mean,month_lag_9_cumsum_auth1_category_1_mean,month_lag_10_cumsum_auth1_category_1_mean,month_lag_11_cumsum_auth1_category_1_mean,month_lag_12_cumsum_auth1_category_1_mean,month_lag_13_cumsum_auth1_category_1_mean,month_lag_14_cumsum_auth1_category_1_mean,month_lag_15_cumsum_auth1_category_1_mean,month_lag_0_cumsum_auth1_category_1_sum,month_lag_1_cumsum_auth1_category_1_sum,month_lag_2_cumsum_auth1_category_1_sum,month_lag_3_cumsum_auth1_category_1_sum,month_lag_4_cumsum_auth1_category_1_sum,month_lag_5_cumsum_auth1_category_1_sum,month_lag_6_cumsum_auth1_category_1_sum,month_lag_7_cumsum_auth1_category_1_sum,month_lag_8_cumsum_auth1_category_1_sum,month_lag_9_cumsum_auth1_category_1_sum,month_lag_10_cumsum_auth1_category_1_sum,month_lag_11_cumsum_auth1_category_1_sum,month_lag_12_cumsum_auth1_category_1_sum,month_lag_13_cumsum_auth1_category_1_sum,month_lag_14_cumsum_auth1_category_1_sum,month_lag_15_cumsum_auth1_category_1_sum,month_lag_0_cumsum_auth1_purchase_amount_mean,month_lag_1_cumsum_auth1_purchase_amount_mean,month_lag_2_cumsum_auth1_purchase_amount_mean,month_lag_3_cumsum_auth1_purchase_amount_mean,month_lag_4_cumsum_auth1_purchase_amount_mean,month_lag_5_cumsum_auth1_purchase_amount_mean,month_lag_6_cumsum_auth1_purchase_amount_mean,month_lag_7_cumsum_auth1_purchase_amount_mean,month_lag_8_cumsum_auth1_purchase_amount_mean,month_lag_9_cumsum_auth1_purchase_amount_mean,month_lag_10_cumsum_auth1_purchase_amount_mean,month_lag_11_cumsum_auth1_purchase_amount_mean,month_lag_12_cumsum_auth1_purchase_amount_mean,month_lag_13_cumsum_auth1_purchase_amount_mean,month_lag_14_cumsum_auth1_purchase_amount_mean,month_lag_15_cumsum_auth1_purchase_amount_mean,month_lag_0_cumsum_auth1_purchase_amount_sum,month_lag_1_cumsum_auth1_purchase_amount_sum,month_lag_2_cumsum_auth1_purchase_amount_sum,month_lag_3_cumsum_auth1_purchase_amount_sum,month_lag_4_cumsum_auth1_purchase_amount_sum,month_lag_5_cumsum_auth1_purchase_amount_sum,month_lag_6_cumsum_auth1_purchase_amount_sum,month_lag_7_cumsum_auth1_purchase_amount_sum,month_lag_8_cumsum_auth1_purchase_amount_sum,month_lag_9_cumsum_auth1_purchase_amount_sum,month_lag_10_cumsum_auth1_purchase_amount_sum,month_lag_11_cumsum_auth1_purchase_amount_sum,month_lag_12_cumsum_auth1_purchase_amount_sum,month_lag_13_cumsum_auth1_purchase_amount_sum,month_lag_14_cumsum_auth1_purchase_amount_sum,month_lag_15_cumsum_auth1_purchase_amount_sum,month_lag_0_cumsum_auth1_diff_days_mean,month_lag_1_cumsum_auth1_diff_days_mean,month_lag_2_cumsum_auth1_diff_days_mean,month_lag_3_cumsum_auth1_diff_days_mean,month_lag_4_cumsum_auth1_diff_days_mean,month_lag_5_cumsum_auth1_diff_days_mean,month_lag_6_cumsum_auth1_diff_days_mean,month_lag_7_cumsum_auth1_diff_days_mean,month_lag_8_cumsum_auth1_diff_days_mean,month_lag_9_cumsum_auth1_diff_days_mean,month_lag_10_cumsum_auth1_diff_days_mean,month_lag_11_cumsum_auth1_diff_days_mean,month_lag_12_cumsum_auth1_diff_days_mean,month_lag_13_cumsum_auth1_diff_days_mean,month_lag_14_cumsum_auth1_diff_days_mean,month_lag_15_cumsum_auth1_diff_days_mean,month_lag_0_cumsum_auth1_installments_mean,month_lag_1_cumsum_auth1_installments_mean,month_lag_2_cumsum_auth1_installments_mean,month_lag_3_cumsum_auth1_installments_mean,month_lag_4_cumsum_auth1_installments_mean,month_lag_5_cumsum_auth1_installments_mean,month_lag_6_cumsum_auth1_installments_mean,month_lag_7_cumsum_auth1_installments_mean,month_lag_8_cumsum_auth1_installments_mean,month_lag_9_cumsum_auth1_installments_mean,month_lag_10_cumsum_auth1_installments_mean,month_lag_11_cumsum_auth1_installments_mean,mon

### Save feature

In [22]:
fname = '151_csm'
ignore_features = ['first_active_month', 'card_id', target]

for col in df_train.columns:
#     if col in ignore_features:
#         continue
#     if not(col.count('feature_')):continue
    if not(col.count('cumsum')):continue
    length = len(df_train[col].dropna())
    if length/len(df_train)<0.1:
        print(col)
        continue
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_train_{col}@', obj=df_train[col].values)
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_test_{col}@', obj=df_test[col].values)